# Import packages

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)
from scipy.stats import levy_stable

from datetime import datetime
from scipy.stats import kstest
from scipy.stats import jarque_bera
# from arch.unitroot import ADF
from scipy.stats import kurtosis
from scipy.stats import skew
# from arch import arch_model

import pickle

import ta

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import webbrowser

plt.style.use('classic')
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 300)

# %config inlinebackend.figure_format = 'svg'

In [2]:
import sys

sys.path.insert(0, 'D:/Intraday_trading')

In [3]:
from src.support import *

# Import and pre-processing data

## Interval data

In [4]:
# interval_data = pd.read_csv('D:/Intraday_trading/Training/Data/XAUUSD_M1.csv', sep = '\t')

In [5]:
# interval_data.columns = ['DATE', 'TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICK_VOL', 'VOL', 'SPREAD']
# interval_data['DATE_TIME'] = pd.to_datetime(interval_data['DATE'] + ' ' + interval_data['TIME'])
# interval_data['DATE'] = pd.to_datetime(interval_data['DATE'])

# interval_data = interval_data[['DATE_TIME', 'DATE', 'TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICK_VOL', 'SPREAD']]

In [6]:
# interval_data

In [7]:
# interval_data.to_pickle('D:/Intraday_trading/Training/Data/XAUUSD_M1.pkl')

In [8]:
%%time

interval_data = pd.read_pickle('D:/Intraday_trading/Training/Data/XAUUSD_M1.pkl')

interval_data = interval_data.set_index('DATE_TIME')
interval_data.index = pd.to_datetime(interval_data.index)

interval_data['DATE'] = pd.to_datetime(interval_data['DATE'])
interval_data['OPEN'] = interval_data['OPEN']
interval_data['HIGH'] = interval_data['HIGH']
interval_data['LOW'] = interval_data['LOW']
interval_data['CLOSE'] = interval_data['CLOSE']


CPU times: total: 93.8 ms
Wall time: 205 ms


In [9]:
interval_data

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,TICK_VOL,SPREAD
DATE_TIME,,,,,,,,
2020-01-02 01:00:00,2020-01-02,01:00:00,1518.464,1518.996,1517.172,1518.896,127,260
2020-01-02 01:01:00,2020-01-02,01:01:00,1518.880,1519.624,1518.871,1519.548,70,260
2020-01-02 01:02:00,2020-01-02,01:02:00,1519.530,1519.535,1518.749,1518.828,59,260
2020-01-02 01:03:00,2020-01-02,01:03:00,1518.845,1518.845,1518.401,1518.485,54,260
2020-01-02 01:04:00,2020-01-02,01:04:00,1518.477,1518.477,1518.084,1518.084,56,260
...,...,...,...,...,...,...,...,...
2024-05-31 10:32:00,2024-05-31,10:32:00,2342.738,2342.867,2341.812,2341.825,85,200
2024-05-31 10:33:00,2024-05-31,10:33:00,2341.828,2342.275,2341.779,2342.145,66,200
2024-05-31 10:34:00,2024-05-31,10:34:00,2342.098,2342.472,2342.089,2342.443,60,200


### Prepare_df

In [10]:
df_1_hour = prepare_df(df = interval_data, timeframe = '1H')

In [11]:
df_1_hour.shape

(26094, 28)

In [12]:
plot_df(df_1_hour.iloc[15000: 15300, :], 
        path = None,# 'D:/Intraday_trading/Training/Saved_results/plot_df.html', 
        open_tab = False)

### Support/ Resistance

In [13]:
df_observe = df_1_hour.iloc[4000: 4500, :].copy()

In [14]:
df_observe.shape

(500, 28)

In [15]:
sr_arr = list(range(5, 30, 5))
pr_arr = [3, 4, 5]
pt_arr = list(range(5, 10, 1))
max_num_arr = [3, 4, 5]
cutoff_arr = [0, 1, 2, 3, 4, 5]

In [16]:
best_params = search_sr(df_observe.copy(), 
                        sr_arr = sr_arr,
                        pr_arr = pr_arr,
                        pt_arr = pt_arr,
                        max_num_arr = max_num_arr,
                        cutoff_arr = cutoff_arr
                       )

100%|██████████| 200/200 [00:07<00:00, 26.01trial/s, best loss: -9.64]            


In [17]:
best_params

{'cutoff': 0,
 'max_num_range': 5,
 'patience_range': 3,
 'patience_time': 5,
 'sr_range': 5}

In [18]:
plot_sr(df_observe = df_observe.copy(),
        sr_range = best_params['sr_range'],
        patience_range = best_params['patience_range'],
        patience_time = best_params['patience_time'],
        max_num_sr = best_params['max_num_range'],
        cutoff = best_params['cutoff'],
        path = None, # 'D:/Intraday_trading/Training/Saved_results/plot_sr.html',
        open_tab = False,
        )

### Extrema

In [19]:
search_extremum(df_observe = df_observe.copy(), order = 30)

(array([  4,  81, 172, 222, 427], dtype=int64),
 array([ 34,  93, 195, 242, 306, 351], dtype=int64))

In [20]:
plot_extremum(df_observe = df_observe.copy(),
              order = 30,
              path = None,
              open_tab = False
)

### Check num data per day

In [ ]:
list_of_dates = sorted(set(interval_data.index.date))

In [ ]:
temp = []

for idx, i in enumerate(list_of_dates[-11:-5]):
    # if interval_data[interval_data.index.date == i].shape != (243, 5):
    #     temp.append(interval_data[interval_data.index.date == i].index.values)
    print(i.weekday())
    print(interval_data[interval_data.index.date == i].shape[0])
    

## Ticks data

In [ ]:
def buy_or_sell(flag):
    '''
    https://www.mql5.com/en/forum/77002
    https://www.mql5.com/en/forum/434077
    '''
    if flag == 2 :
        return 'bid_changed'
    elif flag == 4:
        return 'ask_changed'
    elif flag == 6:
        return 'bid_ask_changed'

In [ ]:
ticks_data = pd.read_csv('D:/Intraday_trading/Training/Data/XAUUSD_ticks.csv', sep = '\t')

In [ ]:
ticks_data.columns = ['DATE', 'TIME', 'BID', 'ASK', 'LAST', 'VOLUME', 'FLAGS']
ticks_data['DATE_TIME'] = pd.to_datetime(ticks_data['DATE'] + ' ' + ticks_data['TIME'])
ticks_data['DATE'] = pd.to_datetime(ticks_data['DATE'])
ticks_data['FLAGS'] = ticks_data['FLAGS'].apply(lambda x: buy_or_sell(x))

ticks_data = ticks_data[['DATE_TIME', 'DATE', 'TIME', 'BID', 'ASK', 'FLAGS']]

In [ ]:
ticks_data.to_pickle('D:/Intraday_trading/Training/Data/XAUUSD_ticks.pkl')

In [ ]:
ticks_data.shape

In [ ]:
# %%time

# ticks_data = pd.read_pickle('D:/Intraday_trading/Training/Data/EURUSD_ticks.pkl')

# ticks_data = ticks_data.set_index('DATE_TIME')
# ticks_data.index = pd.to_datetime(ticks_data.index)

# ticks_data['DATE'] = pd.to_datetime(ticks_data['DATE'])


In [ ]:
# ticks_data.shape

In [ ]:
# ticks_data.head()